In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
df=pd.read_csv("data.csv")

In [ ]:
x1=[]
y1=[]
for ind,x in df.iterrows():
    if(x[0]==1):
        x1.append(x[1:])
        y1.append(x[0])

In [ ]:
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

def SMOTE(X, y, N, k):

    X_synthetic = []
    y_synthetic = []

    for i in range(N):
        random_index = np.random.choice(len(X))
        target_sample = X[random_index]

        distances = np.array([euclidean_distance(target_sample, x) for x in X])
        sorted_indices = np.argsort(distances)
        k_nearest_indices = sorted_indices[1:k+1]
  
        nn_index = np.random.choice(k)
        nn_sample = X[k_nearest_indices[nn_index]]

        rand = np.random.random()

        synthetic_sample = target_sample + rand * (nn_sample - target_sample)
        X_synthetic.append(synthetic_sample)
        y_synthetic.append(y[random_index])

    X_synthetic = np.array(X_synthetic)
    y_synthetic = np.array(y_synthetic)

    return X_synthetic, y_synthetic


In [ ]:
x,y=SMOTE(x1,y1,4000,5)

In [ ]:
for i in range(0,4000):
    temp=[1]
    for j in range(0,len(x[i])):
        temp.append(x[i][j])
    df.loc[len(df.index)] =temp

In [ ]:
n_inputs=len(df)
df=df.sample(frac=1)
train_size=int(0.8*n_inputs)
train_data=df[0:train_size]
train_data=np.array(train_data.values)
test_data=df[train_size:]
test_data=np.array(test_data.values)
y_train=train_data[:, 0]
train_data=train_data[:, 1:]
y_test=test_data[:, 0]
test_data=test_data[:, 1:]
n_features=train_data.shape[1]
test_data_size=len(test_data)
train_data_size=len(train_data)

In [ ]:
#Min-Max standardisation
for i in range(0,n_features):
    mi=1e10
    ma=0
    for j in range(0,train_data_size):
        mi=min(mi,train_data[j][i])
        ma=max(ma,train_data[j][i])
    if(ma==mi):
        continue
    for j in range(0,train_data_size):
        train_data[j][i]-=mi
        train_data[j][i]/=(ma-mi)
for i in range(0,n_features):
    mi=1e10
    ma=0
    for j in range(0,test_data_size):
        mi=min(mi,test_data[j][i])
        ma=max(ma,test_data[j][i])
    if(ma==mi):
        continue
    for j in range(0,test_data_size):
        test_data[j][i]-=mi
        test_data[j][i]/=(ma-mi)

In [ ]:
def sigmoid(x):
    if(x<0):
        return np.exp(x)/(1+np.exp(x))
    else:
        return 1.0/(1.0+np.exp(-x))
    
def sigmoid_derivative(x):
    return sigmoid(x)*(1-sigmoid(x))

def relu(x, alpha=0.01):
    return np.where(x > 0, x, alpha * x)

def relu_derivative(x, alpha=0.01):
    return np.where(x > 0, 1, alpha)

In [ ]:
n_layers=4
n_neurons=[n_features,5,5,1]
activators=["None","Relu","Relu","Sig"]

In [ ]:
#Initialising weights and bias for the layers
weights=[0]
bias=[0]
for i in range(1,n_layers):
    rows=n_neurons[i]
    cols=n_neurons[i-1]
    if(activators[i]=="Relu"):
        #he initialization
        w=np.random.randn(rows,cols)*np.sqrt(2.0/(cols+rows))
        b=np.random.randn(rows,1)*np.sqrt(2.0/(cols+rows))
        weights.append(w)
        bias.append(b)
    else:
        # xavier initialization
        w=np.random.randn(rows,cols)*np.sqrt(6.0/(cols+rows))*4.0
        b=np.random.randn(rows,1)*np.sqrt(6.0/(cols+rows))*4.0
        weights.append(w)
        bias.append(b)

In [ ]:
learning_rate=0.01

In [ ]:
def feedforward(x):
    a=[x]       #holds activation value
    z=[0]       #holds preactivation values
    for i in range(1,n_layers):
        if(activators[i]=="Relu"):
            z.append(np.matmul(weights[i],a[i-1])+bias[i])
            sz=z[i].shape[0]
            temp=np.zeros((sz,1))
            for j in range(0,sz):
                temp[j][0]=relu(z[i][j][0])
            a.append(temp)
        else:
            z.append(np.matmul(weights[i],a[i-1])+bias[i])
            sz=z[i].shape[0]
            temp=np.zeros((sz,1))
            for j in range(0,sz):
                temp[j][0]=sigmoid(z[i][j][0])
            a.append(temp)
    return a,z

In [ ]:
def backpropagate(x,y):
    a,z=feedforward(x)
    pred_value=a[n_layers-1]
    t=(pred_value-y)*relu_derivative(z[n_layers-1])
    deltas=[]
    deltas.append(t)
    i=n_layers-2
    curr=0
    while(i>0):
        rows=z[i].shape[0]
        cols=z[i].shape[1]
        temp=np.zeros((rows,cols))
        for j in range(0,rows):
            for k in range(0,cols):
                if(activators[i]=="Relu"):
                 temp[j][k]=relu_derivative(z[i][j][k])   # temp stores the derivatives of the activation function for each neuron in that layer 
                else:
                 temp[j][k]=sigmoid_derivative(z[i][j][k])
        d=np.array(deltas[curr])
        temp1=np.matmul(np.transpose(weights[i+1]),d)           # calculating error gradient for curr layer using error gradient of next layer
        for j in range(0,rows):
            for k in range(0,cols):
                temp1[j][k]=temp1[j][k]*temp[j][k]           #   calculating delta and stroing it into temp1
        
        deltas.append(temp1)                                #deltas represent the gradient of the loss function with respect to the pre-activation values of the neurons in that layer
        curr+=1
        i-=1
    deltas.append(0)
    deltas.reverse()
    for i in range(0,n_layers-1):
        d=np.array(deltas[i+1])
        grad=np.matmul(d,np.transpose(a[i]))
        weights[i+1]=weights[i+1]-learning_rate*grad
        bias[i+1]=bias[i+1]-learning_rate*d
    

In [ ]:
for k in range(0,10):
    x=np.zeros((n_features,1))
    for i in range(0,train_data_size):
        for j in range(0,n_features):
            x[j][0]=train_data[i][j]
        backpropagate(x,y_train[i])
    count0=0
    count1=0
    count_true0=0
    count_true1=0
    count=0
    count_true=0
    x=np.zeros((n_features,1))
    for i in range(0,test_data_size):
            for j in range(0,n_features):
                x[j][0]=test_data[i][j]
            a,z=feedforward(x)
            if(a[n_layers-1]>=0.50):
                prediction=1
            else:
                prediction=0
            if(prediction==y_test[i]):
                count_true+=1
                if(prediction==1):
                     count_true1+=1
                else:
                     count_true0+=1
            if(y_test[i]==1):
                 count1+=1
            else:
                 count0+=1
            count+=1
    print(count_true/count)

In [ ]:
count=0
count_true=0
c=0
x=np.zeros((n_features,1))
for i in range(0,test_data_size):
    for j in range(0,n_features):
        x[j][0]=test_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
         prediction=0
    if(prediction==0):
        c+=1
    if(prediction==y_test[i] and y_test[i]==1):
        count+=1
    if(y_test[i]==1):
        count_true+=1
print(count/count_true)
    

In [ ]:
count=0
count_true=0
c=0
x=np.zeros((n_features,1))
for i in range(0,train_data_size):
    for j in range(0,n_features):
        x[j][0]=train_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
         prediction=0
    if(prediction==0):
        c+=1
    if(prediction==y_train[i] and y_train[i]==1):
        count+=1
    if(y_train[i]==1):
        count_true+=1
precision_train=count/count_true
print(f'Precision: {precision_train}')

In [ ]:
count1=0
count_true0=0
count_true1=0
count=0
count_true=0
x=np.zeros((n_features,1))
for i in range(0,test_data_size):
    for j in range(0,n_features):
        x[j][0]=test_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
        prediction=0
    if(prediction==y_test[i]):
        count_true+=1
        if(prediction==1):
            count_true1+=1
        else:
            count_true0+=1
    count+=1
acc = count_true/count
print(f'Accuracy test: {acc}')
    

In [ ]:
count1=0
count_true0=0
count_true1=0
count=0
count_true=0
x=np.zeros((n_features,1))
for i in range(0,train_data_size):
    for j in range(0,n_features):
        x[j][0]=train_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
        prediction=0
    if(prediction==y_train[i]):
        count_true+=1
        if(prediction==1):
            count_true1+=1
        else:
            count_true0+=1
    count+=1
acc_train = count_true/count
print(f'Accuracy training: {acc_train}')
    

In [ ]:
tp=0
fn=0
for i in range(0,test_data_size):
    for j in range(0,n_features):
                x[j][0]=test_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
        prediction=0
    if(prediction==1):
        if(y_test[i]==1):
                      tp+=1
        else:
                      fn+=1
    if(prediction==0):
                if(y_test[i]==1):
                      fn+=1
recall=tp/(tp+fn)
print(f'Recall: {tp/(tp+fn)}')

In [ ]:
tp=0
fn=0
for i in range(0,train_data_size):
    for j in range(0,n_features):
                x[j][0]=train_data[i][j]
    a,z=feedforward(x)
    if(a[n_layers-1]>=0.50):
        prediction=1
    else:
        prediction=0
    if(prediction==1):
        if(y_train[i]==1):
                      tp+=1
        else:
                      fn+=1
    if(prediction==0):
                if(y_train[i]==1):
                      fn+=1
recall_train=tp/(tp+fn)
print(f'Recall: {recall_train}')